In [36]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
from preprocessing import preprocess_keypoints

We start with some defenitions

In [37]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# Constants for updating tracking aswell as when to save hand position
update_time = 5
save_time = 5

## Data collection
We use the cv2 library to collect camera fotage and then use mediapipe to get the landmarks on one hand.

To record data press the letter you are currently showing. Data can only be collected if the software is tracking your hand.

In [38]:
# Keep track of data
keypoint_data = []

# For webcam input:
cap = cv2.VideoCapture(1)
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

  i = 0

  # Open camera
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      continue

    # Convert BGR image to RGB and flip the image for selfie camera
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Run mediapipe hands and save the results!
    results = hands.process(image)

    # Convert back to correct colors
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Save key press
    key = cv2.waitKey(update_time)
    i = i + update_time

    if results.multi_hand_landmarks:

        # Pick the first hand
        hand_landmarks = results.multi_hand_landmarks[0]

        # if save time has passed, save the keypoints based on keypress
        if not key == -1 and i >= save_time:
          i = 0

          # Is keypress in the alphabet?
          if chr(key) in list(string.ascii_lowercase):

            # If so save landmarks as an array with values [0,1] (except z axis)
            res = preprocess_keypoints(hand_landmarks)
            res.insert(0, chr(key))
            keypoint_data.append(res)
  
        # Draw on the image
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    
    # exit if space is pressed
    if key == ord(' '):
      break
            
    # Display the image!
    cv2.imshow('MediaPipe Hands', image)
    
cap.release()

## Saving the data
After we have collected all the data we need, we can save it into a csv file with pandas!

In [39]:
# Save the data to a csv file
df = pd.DataFrame(np.array(keypoint_data))
#df.to_csv('data.csv', index = False)
df.to_csv('data.csv', mode='a', header=False)